# Logging, evaluating, and tracing Cerebras models with Braintrust

## Setup

Let's install some dependencies.


%pip install autoevals braintrust openai


Braintrust knows how to intercept calls to the `openai` client library to automatically trace them. Since Cerebras has an OpenAI-compatible API, it's a breeze to set this up!


In [1]:
import os

import openai
import braintrust

client = braintrust.wrap_openai(
    openai.OpenAI(
        api_key=os.getenv("CEREBRAS_API_KEY"),
        base_url="https://api.cerebras.ai/v1",
    )
)

## Logging

To log to Braintrust, simply initialize a logger. All Cerebras model calls will be automatically traced and logged to Braintrust. This works for both streaming and non-streaming calls.


In [3]:
braintrust.init_logger("Cerebras test")

response = client.chat.completions.create(
    model="llama3.1-8b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of the Nevada?"},
    ],
)

print(response.choices[0].message.content)

The capital of Nevada is Carson City.


In Braintrust, we'll see the completion along with a bunch of metrics. Wow, Cerebras is fast!

![Log view](./assets/Log-view.png)


If you enter your Cerebras API key in Braintrust (under Settings -> AI providers), you can also reproduce the call in the UI, and even tweak the prompt!

![Tweak prompt](./assets/Tweak-prompt.gif)

## Evaluating

Evals automatically support Cerebras models as well. Let's run a simple math test eval and see how it does.
